In [1]:
import numpy as np
import pandas as pd
import QuantLib as ql
import scipy.optimize as opt
from IRCurve import SOFR
from Heston import HestonPathGenerator, HestonModelCalibrator
from Snowball import Snowball

In [2]:
# QuantLib Settings
valuationDate = ql.Date(29,4,2022)
ql.Settings.instance().evaluationDate = valuationDate
convention = ql.ModifiedFollowing
dayCounter = ql.Actual360()
calendar = ql.HongKong()

#### Build SOFR Curve

In [3]:
depositRates = ({
    "Tenor": ["1D"],
    "Rate": [0.0028]})
depositRates = pd.DataFrame(depositRates)
depositRates

,Tenor,Rate
0,1D,0.0028


In [4]:
swapRates = ({
    "Tenor": ["1W", "2W", "3W", "1M", "2M", "3M", "4M", "5M", "6M", "7M", "8M", "9M", "10M", "11M", "1Y", "18M", "2Y", "3Y", "4Y", "5Y", 
              "6Y", "7Y", "8Y", "9Y", "10Y"],
    "Rate": [0.0065, 0.007275, 0.007527, 0.007696, 0.009606, 0.011253, 0.013212, 0.014446, 0.01581, 0.017233, 0.018365, 0.019415,
            0.020355, 0.021253, 0.02247, 0.025478, 0.026937, 0.027597, 0.027416, 0.027103, 0.02693, 0.026835, 0.026758, 0.026718, 0.02673]})
swapRates = pd.DataFrame(swapRates)
swapRates

,Tenor,Rate
0,1W,0.006500
1,2W,0.007275
2,3W,0.007527
3,1M,0.007696
4,2M,0.009606
5,3M,0.011253
6,4M,0.013212
7,5M,0.014446
8,6M,0.015810
9,7M,0.017233


In [5]:
sofr_curve = SOFR(depositRates, swapRates, valuationDate)

#### Set Trade Terms

In [6]:
# set trade terms
effectiveDate = ql.Date(27, 4, 2022)
maturityDate = ql.Date(30, 4, 2024)
initialLevel = 417.27
notional = 12130000
inBarrier = 0.8
RedemptionFormula1 = lambda KnockOutDate: notional * (1 + 0.146 * ql.Actual365Fixed().yearFraction(effectiveDate, KnockOutDate) + 0.024)
RedemptionFormula2 = lambda FinalIndexLevel : notional * min(1, FinalIndexLevel / initialLevel) + 0.024
RedemptionFormula3 = notional * (1 + 0.146 * ql.Actual365Fixed().yearFraction(effectiveDate, maturityDate) + 0.024)

In [7]:
# set knock out barrier
outBarrier = ({
    "Observation Date": [ql.Date(26, 10, 2022), ql.Date(25, 11, 2022), ql.Date(26, 12, 2022), ql.Date(20, 1, 2023), ql.Date(24, 2, 2023),
                        ql.Date(24, 3, 2023), ql.Date(26, 4, 2023), ql.Date(26, 5, 2023), ql.Date(26, 6, 2023), ql.Date(26, 7, 2023),
                        ql.Date(25, 8, 2023), ql.Date(26, 9, 2023), ql.Date(26, 10, 2023), ql.Date(24, 11, 2023), ql.Date(26, 12, 2023),
                        ql.Date(26, 1, 2024), ql.Date(26, 2, 2024), ql.Date(26, 3, 2024), ql.Date(26, 4, 2024)],
    "Barrier": [1.03, 1.025, 1.02, 1.015, 1.01, 1.005, 1, 0.995, 0.99, 0.985, 0.98, 0.975, 0.97, 0.965, 0.96, 0.955, 0.95, 0.945, 0.94]})
outBarrier = pd.DataFrame(outBarrier)
outBarrier = outBarrier.set_index(["Observation Date"])
outBarrier

,Barrier
Observation Date,
"October 26th, 2022",1.030
"November 25th, 2022",1.025
"December 26th, 2022",1.020
"January 20th, 2023",1.015
"February 24th, 2023",1.010
"March 24th, 2023",1.005
"April 26th, 2023",1.000
"May 26th, 2023",0.995
"June 26th, 2023",0.990


In [8]:
# set past fixings
pastFixings =({
    "Date": [ql.Date(29, 4, 2022), ql.Date(28, 4, 2022), ql.Date(27, 4, 2022)],
    "Fixing": [412, 427.81, 417.27]})
pastFixings = pd.DataFrame(pastFixings)
pastFixings = pastFixings.set_index(["Date"])
pastFixings

,Fixing
Date,
"April 29th, 2022",412.00
"April 28th, 2022",427.81
"April 27th, 2022",417.27


In [9]:
# create discounting curve and dividend curve, required for Heston model
curveHandle = sofr_curve.curveHandle
dividendHandle = ql.YieldTermStructureHandle(ql.FlatForward(valuationDate, 0.0, dayCounter))

#### Heston Process Calibration

In [10]:
# volatility surface data
expiration_dates = [ql.Date(31, 5, 2022), ql.Date(30, 6, 2022), ql.Date(30, 9, 2022), ql.Date(30, 12, 2022), ql.Date(31, 3, 2023),
                    ql.Date(16, 6, 2023), ql.Date(15, 12, 2023), ql.Date(19, 1, 2024), ql.Date(20, 12, 2024)]

strikes = [330.008, 371.259, 391.884, 402.197, 412.51, 422.823, 433.136, 453.761]

data = [[0.4723, 0.3650, 0.3190, 0.2974, 0.2760, 0.2529, 0.2272, 0.1965],
       [0.4115, 0.3339, 0.3001, 0.2838, 0.2673, 0.2496, 0.2303, 0.1950],
       [0.3573, 0.3090, 0.2825, 0.2691, 0.2561, 0.2434, 0.2309, 0.2046],
       [0.3308, 0.2912, 0.2708, 0.2599, 0.2487, 0.2373, 0.2263, 0.2065],
       [0.3186, 0.2835, 0.2667, 0.2579, 0.2487, 0.2389, 0.2287, 0.2091],
       [0.3069, 0.2772, 0.2613, 0.2529, 0.2441, 0.2352, 0.2261, 0.2091],
       [0.2898, 0.2646, 0.2515, 0.2445, 0.2373, 0.2299, 0.2223, 0.2077],
       [0.2865, 0.2619, 0.2493, 0.2428, 0.2361, 0.2292, 0.2223, 0.2089],
       [0.2711, 0.2522, 0.2425, 0.2374, 0.2322, 0.2269, 0.2216, 0.2112]]

In [11]:
# initial parameters for Heston model
theta = 0.01
kappa = 0.01
sigma = 0.01
rho = 0.01
v0 = 0.01

# bounds for model parameters (1=theta, 2=kappa, 3=sigma, 4=rho, 5=v0)
bounds = [(0.01, 1.0), (0.01, 10.0), (0.01, 1.0), (-1.0, 1.0), (0.01, 1.0)]

In [12]:
# calibrate Heston model, print calibrated parameters
calibrationResult = HestonModelCalibrator(valuationDate, calendar, initialLevel, curveHandle, dividendHandle,
                                          v0, kappa, theta, sigma, rho, expiration_dates, strikes, data, opt.differential_evolution, bounds)
print('calibrated Heston parameters', calibrationResult[1].params())

calibrated Heston parameters [ 0.0893722; 0.880817; 0.801844; -1; 0.0853334 ]


#### Monte Carlo Simulation and Valuation

In [13]:
# monte carlo parameter
nPaths = 10000

In [14]:
# request and print PV
test_product = Snowball(effectiveDate, maturityDate, initialLevel, notional, inBarrier, outBarrier,
                        RedemptionFormula1, RedemptionFormula2, RedemptionFormula3,
                        calendar, dayCounter, calibrationResult[0], HestonPathGenerator, nPaths, curveHandle)
value = test_product.valuation(valuationDate, pastFixings)
print(value)

11833187.994327324
